In [1254]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import metrics
from scipy import stats
import statsmodels.api as sm
import math
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sys
import pandas.core.algorithms as algos
pd.pandas.set_option('display.max_columns',None)
from ast import literal_eval

In [1255]:
train=pd.read_csv('train_go05W65.csv', converters={'Product_Holding_B1': literal_eval,'Product_Holding_B2': literal_eval})
test=pd.read_csv('test_VkM91FT.csv', converters={'Product_Holding_B1': literal_eval})

In [1256]:
train.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8]
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3]
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00]
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6]
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]"


In [1257]:
test.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1
0,CC372708,Female,31,31,0,C2,S3,"[P12, P13]"
1,CC216072,Male,28,37,1,C1,S2,"[P12, P13]"
2,CC387629,Male,31,12,0,C2,S3,[P20]
3,CC389228,Female,55,11,0,C2,S2,"[P13, P21]"
4,CC394445,Male,51,49,1,C2,S1,[P13]


In [1258]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37748 entries, 0 to 37747
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer_ID         37748 non-null  object
 1   Gender              37748 non-null  object
 2   Age                 37748 non-null  int64 
 3   Vintage             37748 non-null  int64 
 4   Is_Active           37748 non-null  int64 
 5   City_Category       37748 non-null  object
 6   Customer_Category   37748 non-null  object
 7   Product_Holding_B1  37748 non-null  object
 8   Product_Holding_B2  37748 non-null  object
dtypes: int64(3), object(6)
memory usage: 2.6+ MB


In [1259]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
s=train['Product_Holding_B1']
s1=train['Product_Holding_B2']
B1_dummies=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=train.index).add_prefix('B1_')
B2_dummies=pd.DataFrame(mlb.fit_transform(s1),columns=mlb.classes_, index=train.index).add_prefix('B2_')

In [1260]:
train=pd.concat([train,B1_dummies,B2_dummies],axis=1)

In [1261]:
train.columns

Index(['Customer_ID', 'Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'Product_Holding_B1', 'Product_Holding_B2',
       'B1_P00', 'B1_P1', 'B1_P10', 'B1_P11', 'B1_P12', 'B1_P13', 'B1_P14',
       'B1_P15', 'B1_P16', 'B1_P17', 'B1_P18', 'B1_P19', 'B1_P2', 'B1_P20',
       'B1_P21', 'B1_P3', 'B1_P4', 'B1_P5', 'B1_P6', 'B1_P7', 'B1_P8', 'B1_P9',
       'B2_P00', 'B2_P1', 'B2_P10', 'B2_P11', 'B2_P12', 'B2_P13', 'B2_P14',
       'B2_P15', 'B2_P16', 'B2_P17', 'B2_P18', 'B2_P2', 'B2_P20', 'B2_P3',
       'B2_P4', 'B2_P5', 'B2_P6', 'B2_P7', 'B2_P8', 'B2_P9'],
      dtype='object')

In [1262]:
train.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P14,B2_P15,B2_P16,B2_P17,B2_P18,B2_P2,B2_P20,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3],0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]",0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [1263]:
s=test['Product_Holding_B1']
B1_dummies=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=test.index).add_prefix('B1_')
test=pd.concat([test,B1_dummies],axis=1)

In [1264]:
test.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9
0,CC372708,Female,31,31,0,C2,S3,"[P12, P13]",0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,CC216072,Male,28,37,1,C1,S2,"[P12, P13]",0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,CC387629,Male,31,12,0,C2,S3,[P20],0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,CC389228,Female,55,11,0,C2,S2,"[P13, P21]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,CC394445,Male,51,49,1,C2,S1,[P13],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [1265]:
test.shape

(20327, 30)

In [1266]:
df=pd.concat([train,test],axis='rows')

In [1267]:
df

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P14,B2_P15,B2_P16,B2_P17,B2_P18,B2_P2,B2_P20,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3],0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]",0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20322,CC303542,Female,37,11,1,C2,S3,[P16],NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20323,CC266713,Male,29,14,0,C2,S2,[P16],NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20324,CC393639,Male,26,12,0,C2,S3,[P13],NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20325,CC285013,Female,44,19,1,C1,S3,"[P8, P9, P16]",NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1268]:
B1_features=[feature for feature in df.columns if 'B1' in feature]
B1_features

['Product_Holding_B1',
 'B1_P00',
 'B1_P1',
 'B1_P10',
 'B1_P11',
 'B1_P12',
 'B1_P13',
 'B1_P14',
 'B1_P15',
 'B1_P16',
 'B1_P17',
 'B1_P18',
 'B1_P19',
 'B1_P2',
 'B1_P20',
 'B1_P21',
 'B1_P3',
 'B1_P4',
 'B1_P5',
 'B1_P6',
 'B1_P7',
 'B1_P8',
 'B1_P9']

In [1269]:
B1_features=B1_features[1:]
# B1_features

In [1270]:
df['count_prod']=df[B1_features].sum(axis=1)

In [1271]:
df

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P14,B2_P15,B2_P16,B2_P17,B2_P18,B2_P2,B2_P20,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9,count_prod
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3],0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]",0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20322,CC303542,Female,37,11,1,C2,S3,[P16],NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
20323,CC266713,Male,29,14,0,C2,S2,[P16],NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
20324,CC393639,Male,26,12,0,C2,S3,[P13],NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
20325,CC285013,Female,44,19,1,C1,S3,"[P8, P9, P16]",NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [1272]:
# df["Product_Holding_B1"].value_counts().to_list()

In [1273]:
A=df["Product_Holding_B1"].to_list()
A[0][0][1:]

'16'

In [1274]:
S=[]
for i in range(len(A)):
    c=1
    for j in range(len(A[i])):
        c=c*int(A[i][j][1:])
    S.append(c)
        
        

In [1275]:
len(S)

58075

In [1276]:
# df['SUM_PRODUCT']=S

In [1277]:
# df['temp']=df['Product_Holding_B1'].astype('str')

In [1278]:
# f=['Product_Holding_B1']
# temp=df.groupby('temp')['temp'].count()/len(df)
# temp_df=temp[temp>0.01].index
# df['temp']=np.where(df['temp'].isin(temp_df),df['temp'],'Rare_var')

In [1279]:
# df['temp'].value_counts()

In [1280]:
df

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P14,B2_P15,B2_P16,B2_P17,B2_P18,B2_P2,B2_P20,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9,count_prod
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3],0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]",0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20322,CC303542,Female,37,11,1,C2,S3,[P16],NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
20323,CC266713,Male,29,14,0,C2,S2,[P16],NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
20324,CC393639,Male,26,12,0,C2,S3,[P13],NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
20325,CC285013,Female,44,19,1,C1,S3,"[P8, P9, P16]",NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [1281]:
df['CUS_CAT']=df['Customer_ID'].str[2:3].astype(int)

## label encoding based on count

In [1282]:
cat_features=['Customer_Category','Gender','City_Category']

In [1283]:
df2=df.copy()

In [1284]:
df['Age_str']=df['Age'].astype(str)
df['vint_str']=df['Vintage'].astype(str)

In [1285]:
A=['Age_str','vint_str']+cat_features

In [1286]:
# labelencoder = LabelEncoder()
# for var in A:
#     df[var] = labelencoder.fit_transform(df[var])

In [1287]:
for feature in A:
    dic=df[feature].value_counts().to_dict()
    list1=np.arange(len(dic.values()),0,-1)
    dic2=dict(zip(list(dic.keys()),list1))
    df[feature]=df[feature].replace(dic2)

## One hot encoding

In [1288]:
def category_onehot_multcols(multcolumns):
    
    df_final=df
    i=0
    for fields in multcolumns:
        
        print(fields)
        df1=pd.get_dummies(df[fields],drop_first=True)
        
        df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:
            
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1
       
        
    df_final=pd.concat([df,df_final],axis=1)
        
    return df_final

In [1289]:
# df=category_onehot_multcols(cat_features)

In [1290]:
df

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P14,B2_P15,B2_P16,B2_P17,B2_P18,B2_P2,B2_P20,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9,count_prod,CUS_CAT,Age_str,vint_str
0,CC264719,2,41,14,0,2,3,[P16],[P8],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,2,18,61
1,CC209679,1,47,14,1,2,2,"[P13, P20]",[P3],0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,25,61
2,CC319633,1,59,14,0,1,2,[P11],[P00],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,2,61
3,CC231413,1,32,16,0,2,2,"[P8, P13]",[P6],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2,2,30,62
4,CC259633,2,30,15,0,1,3,"[P16, P17, P21]","[P8, P12]",0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,2,33,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20322,CC303542,1,37,11,1,1,3,[P16],NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,10,60
20323,CC266713,2,29,14,0,1,2,[P16],NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,35,61
20324,CC393639,2,26,12,0,1,3,[P13],NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,32,65
20325,CC285013,1,44,19,1,2,3,"[P8, P9, P16]",NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,27,56


In [1291]:
df['CC_NUM']=df['Customer_ID'].str[2:].astype(int)

In [1292]:
df.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P14,B2_P15,B2_P16,B2_P17,B2_P18,B2_P2,B2_P20,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9,count_prod,CUS_CAT,Age_str,vint_str,CC_NUM
0,CC264719,2,41,14,0,2,3,[P16],[P8],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,2,18,61,264719
1,CC209679,1,47,14,1,2,2,"[P13, P20]",[P3],0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,25,61,209679
2,CC319633,1,59,14,0,1,2,[P11],[P00],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,2,61,319633
3,CC231413,1,32,16,0,2,2,"[P8, P13]",[P6],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2,2,30,62,231413
4,CC259633,2,30,15,0,1,3,"[P16, P17, P21]","[P8, P12]",0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,2,33,63,259633


In [1293]:
feat=['Age','Vintage','Age_str','vint_str','CC_NUM']

In [1294]:
sc = StandardScaler()
df[feat]=sc.fit_transform(df[feat])

In [1295]:
df.head(3)

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P14,B2_P15,B2_P16,B2_P17,B2_P18,B2_P2,B2_P20,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9,count_prod,CUS_CAT,Age_str,vint_str,CC_NUM
0,CC264719,2,0.251198,-0.54482,0,2,3,[P16],[P8],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,2,-0.453761,0.495148,-0.786374
1,CC209679,1,0.844613,-0.54482,1,2,2,"[P13, P20]",[P3],0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,0.221706,0.495148,-1.633929
2,CC319633,1,2.031444,-0.54482,0,1,2,[P11],[P00],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,-1.997684,0.495148,0.059241


In [1296]:
holdings=df[['Product_Holding_B1','Product_Holding_B2']]

In [1297]:
df=df.drop(columns=holdings.columns)

In [1298]:
df.head(2)

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P14,B2_P15,B2_P16,B2_P17,B2_P18,B2_P2,B2_P20,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9,count_prod,CUS_CAT,Age_str,vint_str,CC_NUM
0,CC264719,2,0.251198,-0.54482,0,2,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,2,-0.453761,0.495148,-0.786374
1,CC209679,1,0.844613,-0.54482,1,2,2,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,0.221706,0.495148,-1.633929


In [1299]:
df.drop(columns=['B2_P20','B2_P18','B2_P14','B2_P17'],inplace=True)

In [1300]:
B2_features=[feature for feature in df.columns if'B2' in feature]

In [1301]:
df[B1_features].sum()

B1_P00     1510
B1_P1        15
B1_P10     2246
B1_P11     2937
B1_P12     9443
B1_P13    26072
B1_P14      203
B1_P15      862
B1_P16    17327
B1_P17    11031
B1_P18      533
B1_P19     3384
B1_P2      1000
B1_P20     5701
B1_P21     9817
B1_P3        98
B1_P4        55
B1_P5        51
B1_P6       165
B1_P7       593
B1_P8      4150
B1_P9       724
dtype: int64

In [1302]:
df[B2_features].sum()

B2_P00     5908.0
B2_P1      3310.0
B2_P10     2914.0
B2_P11      186.0
B2_P12     7285.0
B2_P13     1494.0
B2_P15       41.0
B2_P16     3478.0
B2_P2       206.0
B2_P3      2486.0
B2_P4      2943.0
B2_P5      1211.0
B2_P6      4675.0
B2_P7      2526.0
B2_P8     11518.0
B2_P9      1833.0
dtype: float64

In [1303]:
df['B2_P1'].sum()

3310.0

In [1304]:
len(B2_features)

16

In [1305]:
cust_ID=df['Customer_ID']
df=df.drop(columns='Customer_ID')

In [1306]:
df.head()

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P15,B2_P16,B2_P2,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9,count_prod,CUS_CAT,Age_str,vint_str,CC_NUM
0,2,0.251198,-0.544820,0,2,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,2,-0.453761,0.495148,-0.786374
1,1,0.844613,-0.544820,1,2,2,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,0.221706,0.495148,-1.633929
2,1,2.031444,-0.544820,0,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,-1.997684,0.495148,0.059241
3,1,-0.638925,-0.349742,0,2,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2,2,0.704182,0.588157,-1.299249
4,2,-0.836730,-0.447281,0,1,3,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,2,0.993667,0.681166,-0.864692


In [1307]:
# df['CC_NUM']=np.log(df['CC_NUM'])

In [1308]:
scale_features=[feature for feature in df.columns if feature not in feat ]
scale_features

['Gender',
 'Is_Active',
 'City_Category',
 'Customer_Category',
 'B1_P00',
 'B1_P1',
 'B1_P10',
 'B1_P11',
 'B1_P12',
 'B1_P13',
 'B1_P14',
 'B1_P15',
 'B1_P16',
 'B1_P17',
 'B1_P18',
 'B1_P19',
 'B1_P2',
 'B1_P20',
 'B1_P21',
 'B1_P3',
 'B1_P4',
 'B1_P5',
 'B1_P6',
 'B1_P7',
 'B1_P8',
 'B1_P9',
 'B2_P00',
 'B2_P1',
 'B2_P10',
 'B2_P11',
 'B2_P12',
 'B2_P13',
 'B2_P15',
 'B2_P16',
 'B2_P2',
 'B2_P3',
 'B2_P4',
 'B2_P5',
 'B2_P6',
 'B2_P7',
 'B2_P8',
 'B2_P9',
 'count_prod',
 'CUS_CAT']

In [1309]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df[scale_features]=scaler.fit_transform(df[scale_features])

In [1310]:
df.head(2)

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P15,B2_P16,B2_P2,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9,count_prod,CUS_CAT,Age_str,vint_str,CC_NUM
0,1.0,0.251198,-0.54482,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.5,-0.453761,0.495148,-0.786374
1,0.0,0.844613,-0.54482,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.5,0.221706,0.495148,-1.633929


## Model

In [ ]:
pred_features=[feature for feature in df.columns if 'B2' in feature]
pred_features

In [1144]:
df_train=df.iloc[:train.shape[0],:]
df_test=df.iloc[train.shape[0]:,:]

In [1145]:
df_test=df_test.drop(columns=pred_features)
df_test.head(2)

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,count_prod,SUM_PRODUCT,temp,Age_str,vint_str,CC_NUM
0,0.0,-0.737827,1.113339,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,2.376682e-08,-0.540436,0.800677,-1.272030,0.907316
1,1.0,-1.034535,1.698571,1.0,1.0,0.5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,2.376682e-08,-0.540436,1.283153,-1.644067,0.120782


In [1146]:
train_features=[feature for feature in df.columns if feature not in pred_features]

In [1147]:
X_train=df_train[train_features]
y_train=df_train[pred_features]

In [1148]:
X_train.head(2)

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P00,B1_P1,B1_P10,B1_P11,B1_P12,B1_P13,B1_P14,B1_P15,B1_P16,B1_P17,B1_P18,B1_P19,B1_P2,B1_P20,B1_P21,B1_P3,B1_P4,B1_P5,B1_P6,B1_P7,B1_P8,B1_P9,count_prod,SUM_PRODUCT,temp,Age_str,vint_str,CC_NUM
0,1.0,0.251198,-0.54482,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.437622e-09,0.511062,-0.453761,0.495148,0.413732
1,0.0,0.844613,-0.54482,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,3.961136e-08,-2.292934,0.221706,0.495148,0.077453


In [1149]:
y_train.head(2)

,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P15,B2_P16,B2_P2,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1150]:
y_train.sum(axis=0)

B2_P00     5908.0
B2_P1      3310.0
B2_P10     2914.0
B2_P11      186.0
B2_P12     7285.0
B2_P13     1494.0
B2_P15       41.0
B2_P16     3478.0
B2_P2       206.0
B2_P3      2486.0
B2_P4      2943.0
B2_P5      1211.0
B2_P6      4675.0
B2_P7      2526.0
B2_P8     11518.0
B2_P9      1833.0
dtype: float64

In [901]:
from sklearn.neighbors import KNeighborsClassifier

In [902]:
knn_clf = KNeighborsClassifier()

In [903]:
# knn_clf.fit(X_train, y_train)

KNeighborsClassifier()

In [904]:
# pred=knn_clf.predict_proba(df_test)

In [1151]:
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import GaussianNB

In [1152]:
xgb_estimator = LogisticRegression(fit_intercept=False, C=1e12)
# xgb_estimator = GaussianNB()
# xgb_estimator =xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.4, gamma=0.4, gpu_id=-1,
#               importance_type='gain', interaction_constraints='',
#               learning_rate=0.3, max_delta_step=0, max_depth=12,
#               min_child_weight=7, missing=None, monotone_constraints='()',
#               n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
#               tree_method='exact', validate_parameters=1, verbosity=None)

In [1153]:
multilabel_model = MultiOutputClassifier(xgb_estimator)

In [1154]:
multilabel_model.fit(X_train, y_train)

C:\Users\461866\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\461866\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

MultiOutputClassifier(estimator=LogisticRegression(C=1000000000000.0,
                                                   fit_intercept=False))

In [1155]:
# params={
#  "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
#  "max_depth"        : [10, 12, 15,20,25,30],
#  "min_child_weight" : [ 1, 3, 5, 7 ],
#  "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
#  "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]  
# }

In [1156]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# import xgboost

# def timer(start_time=None):
#     if not start_time:
#         start_time = datetime.now()
#         return start_time
#     elif start_time:
#         thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
#         tmin, tsec = divmod(temp_sec, 60)
#         print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [1157]:
# classifier=MultiOutputClassifier(xgboost.XGBClassifier())

In [1158]:
# random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',cv=5,verbose=3)

# from datetime import datetime
# start_time = timer(None) # timing starts from this point for "start_time" variable
# random_search.fit(X_train,y_train)
# timer(start_time) # timing ends here for "start_time" variable

In [1159]:
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_train, multilabel_model.predict(X_train))*100))

Accuracy on test data: 46.2%


In [1160]:
pred_train=multilabel_model.predict_proba(X_train)

In [1161]:
pred_test=multilabel_model.predict_proba(df_test)

In [1162]:
pred_train[0][:,1]

array([5.03650869e-04, 5.01859181e-12, 8.84975839e-01, ...,
       9.85350064e-01, 1.20916076e-06, 5.67451791e-12])

In [1163]:
pred_test[0][:,1]

array([1.34378225e-06, 8.66772098e-11, 2.02138987e-01, ...,
       6.18170322e-01, 7.78194206e-10, 3.48567174e-17])

In [1164]:
def cutoff_youdens_j(fpr,tpr,thresholds):
    a = tpr-fpr
    b = sorted(zip(a,thresholds))
    return b[-1][1]

In [1165]:
fpr, tpr, threshold = metrics.roc_curve(y_train.iloc[:,0], pred_train[0][:,1])

In [1166]:
# len(threshold)

In [1167]:
cutoff = cutoff_youdens_j(fpr, tpr, threshold)
cutoff

0.3118058468019044

In [1168]:
def accuray_cutoff(E,F,thresholds):
    A=[]
    B=[]
    for k in range(len(thresholds)):
        cuto=thresholds[k]
        predicted = pd.DataFrame()
        predicted= F
        predicted= np.where(predicted > float(cuto), 1, 0)
        precision_train = metrics.precision_score(E, predicted)
        A.append(precision_train)
        B.append(cuto)
    A=pd.DataFrame(A,columns=['precision'])
    B=pd.DataFrame(B,columns=['cutoff'])
    C=pd.concat([A,B],axis='columns')
    C.sort_values(by='precision', ascending=False, inplace=True)
    x=float(C.iloc[0,1])
        
    return(x)

In [1169]:
# accuray_cutoff(y_train.iloc[:,0], pred_train[0][:,1],threshold)

In [1170]:
# for i in range(18):
#     A=pred_train[i][:,1]
#     B=pred_test[i][:,1]
#     fpr, tpr, threshold = metrics.roc_curve(y_train.iloc[:,i], pred_train[i][:,1])
    
    
    

In [1171]:
ID_train=pd.DataFrame(cust_ID.iloc[:train.shape[0]],columns=['Customer_ID'])
ID_test=pd.DataFrame(cust_ID.iloc[train.shape[0]:],columns=['Customer_ID'])
proba_test=pd.DataFrame(cust_ID.iloc[train.shape[0]:],columns=['Customer_ID'])

In [1172]:
cut=[]

In [1173]:
for i in range(16):
    A=pred_train[i][:,1]
    B=pred_test[i][:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_train.iloc[:,i], pred_train[i][:,1])
    cutoff = cutoff_youdens_j(fpr, tpr, threshold)
#     cutoff=accuray_cutoff(y_train.iloc[:,i],pred_train[i][:,1],threshold)
    cut.append(cutoff)
    predicted = pd.DataFrame()
    predicted= B
    predicted= np.where(predicted > float(cutoff), 1, 0)
    predicted=pd.DataFrame(predicted,columns=[i])
    ID_test=pd.concat([ID_test,predicted],axis='columns')
    predicted_proba=B
    predicted_proba=pd.DataFrame(predicted_proba,columns=[i])
    proba_test=pd.concat([proba_test,predicted_proba],axis='columns')

In [1174]:
test_ID=ID_test['Customer_ID']

In [1175]:
ID_test.drop(columns=['Customer_ID'],inplace=True)
proba_test.drop(columns=['Customer_ID'],inplace=True)

In [1176]:
ID_test.rename(columns=dict(zip(ID_test.columns,y_train.columns)),inplace=True)
proba_test.rename(columns=dict(zip(proba_test.columns,y_train.columns)),inplace=True)

In [1177]:
ID_test=pd.concat([test_ID,ID_test],axis='columns')
proba_test=pd.concat([test_ID,proba_test],axis='columns')

In [1178]:
ID_test.head(1)

,Customer_ID,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P15,B2_P16,B2_P2,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9
0,CC372708,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1


In [1179]:
proba_test

,Customer_ID,B2_P00,B2_P1,B2_P10,B2_P11,B2_P12,B2_P13,B2_P15,B2_P16,B2_P2,B2_P3,B2_P4,B2_P5,B2_P6,B2_P7,B2_P8,B2_P9
0,CC372708,1.343782e-06,0.088033,4.417261e-01,4.493681e-21,1.179958e-41,2.502417e-43,3.788527e-89,4.375968e-101,6.231476e-11,0.000000e+00,0.287808,0.099531,0.026002,0.039997,7.067078e-02,7.308719e-02
1,CC216072,8.667721e-11,0.105059,4.115004e-02,9.056268e-21,4.904438e-43,1.459117e-43,1.000229e-88,2.090794e-103,7.945201e-11,4.602541e-46,0.027481,0.003500,0.106901,0.076317,9.163708e-01,4.602650e-02
2,CC387629,2.021390e-01,0.012997,1.485685e-02,1.775989e-02,2.186958e-02,1.950549e-01,5.811763e-04,8.075171e-01,2.143612e-03,0.000000e+00,0.008217,0.002059,0.003890,0.003199,9.746632e-03,9.110851e-03
3,CC389228,8.504345e-08,0.048113,2.379094e-01,3.990094e-21,7.940343e-01,6.910603e-31,1.775808e-50,1.060907e-57,2.024263e-09,0.000000e+00,0.153860,0.097118,0.033094,0.018134,5.057455e-02,9.150163e-02
4,CC394445,7.838260e-01,0.244752,1.356335e-02,1.397356e-24,1.886709e-01,1.367683e-28,1.379620e-50,2.402636e-61,7.251761e-13,0.000000e+00,0.016020,0.009368,0.004781,0.007552,1.248050e-02,1.507542e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20322,CC303542,7.097472e-03,0.071734,2.042383e-02,1.912704e-26,1.855071e-02,1.392110e-21,5.428157e-03,8.387537e-75,3.440423e-09,0.000000e+00,0.012142,0.004154,0.081171,0.016889,3.720491e-01,1.312950e-02
20323,CC266713,5.844634e-04,0.003564,1.708930e-02,1.003993e-25,3.894430e-02,1.239800e-21,2.773751e-03,4.499522e-75,1.462728e-08,0.000000e+00,0.010711,0.002097,0.134678,0.020863,6.578008e-01,1.759307e-02
20324,CC393639,6.181703e-01,0.021499,6.937603e-02,1.308029e-20,1.339852e-01,4.334304e-31,1.938797e-50,1.113787e-57,3.584063e-08,0.000000e+00,0.037942,0.015559,0.035786,0.010185,1.987318e-02,8.673709e-02
20325,CC285013,7.781942e-10,0.910390,1.166531e-32,5.674863e-48,5.511986e-37,6.077429e-45,1.050816e-50,3.156840e-156,3.160871e-07,0.000000e+00,0.259853,0.077846,0.711849,0.647455,3.891177e-10,2.216252e-26


In [1180]:
cut

[0.3118058468019044,
 0.0857601627235825,
 0.0685457962558077,
 0.005103352199998507,
 0.11588769089196932,
 0.005205565572678925,
 0.0009032526938273556,
 0.09990260753857545,
 0.004144817008707082,
 0.7030669732338515,
 0.06672116878787969,
 0.023904737023278517,
 0.07731959765506638,
 0.08287895914982618,
 0.2153431903890526,
 0.05239836303613167]

In [1181]:
S=[]
for i in range(ID_test.shape[0]):
    A=ID_test.iloc[i,1:].to_list()
    B=proba_test.iloc[i,1:].to_list()
    C=[]
    D=[]
    for j in range(0,len(A)):
        if A[j]==1:
            C.append(ID_test.iloc[:,j+1:j+2].columns.to_list()[0])
            D.append(B[j]-cut[j])
    new_dict = dict(zip(C, D))
    X={k: v for k, v in sorted(new_dict.items(), key=lambda item: item[1],reverse=True)}
    newlist = list()
    for k in X.keys():
        newlist.append(k)
    S.append(newlist)
    
            

In [1182]:
S

[['B2_P10', 'B2_P4', 'B2_P5', 'B2_P9', 'B2_P1'],
 ['B2_P8', 'B2_P6', 'B2_P1'],
 ['B2_P16', 'B2_P13', 'B2_P11'],
 ['B2_P12', 'B2_P10', 'B2_P4', 'B2_P5', 'B2_P9'],
 ['B2_P00', 'B2_P1', 'B2_P12'],
 ['B2_P8', 'B2_P12', 'B2_P6', 'B2_P9'],
 ['B2_P16', 'B2_P13', 'B2_P2', 'B2_P11'],
 ['B2_P8', 'B2_P12', 'B2_P6', 'B2_P15'],
 ['B2_P8', 'B2_P6', 'B2_P15'],
 ['B2_P4', 'B2_P8', 'B2_P5', 'B2_P6', 'B2_P7', 'B2_P9'],
 ['B2_P8',
  'B2_P9',
  'B2_P1',
  'B2_P7',
  'B2_P3',
  'B2_P6',
  'B2_P10',
  'B2_P4',
  'B2_P5'],
 ['B2_P16', 'B2_P13', 'B2_P15'],
 ['B2_P8', 'B2_P10', 'B2_P1', 'B2_P6', 'B2_P7', 'B2_P4', 'B2_P5'],
 ['B2_P00', 'B2_P2', 'B2_P6'],
 ['B2_P8', 'B2_P6', 'B2_P15'],
 ['B2_P12', 'B2_P10', 'B2_P5', 'B2_P15'],
 ['B2_P8', 'B2_P7', 'B2_P6', 'B2_P10', 'B2_P3', 'B2_P9', 'B2_P4', 'B2_P5'],
 ['B2_P6', 'B2_P7', 'B2_P4', 'B2_P1', 'B2_P5'],
 ['B2_P8', 'B2_P6', 'B2_P9'],
 ['B2_P00', 'B2_P12'],
 ['B2_P00', 'B2_P1', 'B2_P12'],
 ['B2_P00', 'B2_P12'],
 ['B2_P16', 'B2_P13', 'B2_P11'],
 ['B2_P8', 'B2_P6', 'B2_P

In [1183]:
for i in range(len(S)):
    if len(S[i])>3:
        S[i]=S[i][:3]

In [1184]:
for i in range(len(S)):
    for j in range(len(S[i])):
        S[i][j]=S[i][j][3:]
        

In [1185]:
S

[['P10', 'P4', 'P5'],
 ['P8', 'P6', 'P1'],
 ['P16', 'P13', 'P11'],
 ['P12', 'P10', 'P4'],
 ['P00', 'P1', 'P12'],
 ['P8', 'P12', 'P6'],
 ['P16', 'P13', 'P2'],
 ['P8', 'P12', 'P6'],
 ['P8', 'P6', 'P15'],
 ['P4', 'P8', 'P5'],
 ['P8', 'P9', 'P1'],
 ['P16', 'P13', 'P15'],
 ['P8', 'P10', 'P1'],
 ['P00', 'P2', 'P6'],
 ['P8', 'P6', 'P15'],
 ['P12', 'P10', 'P5'],
 ['P8', 'P7', 'P6'],
 ['P6', 'P7', 'P4'],
 ['P8', 'P6', 'P9'],
 ['P00', 'P12'],
 ['P00', 'P1', 'P12'],
 ['P00', 'P12'],
 ['P16', 'P13', 'P11'],
 ['P8', 'P6', 'P15'],
 ['P8', 'P6', 'P15'],
 ['P6', 'P7', 'P1'],
 ['P12', 'P10', 'P9'],
 ['P8', 'P3', 'P6'],
 ['P8', 'P4', 'P6'],
 ['P00', 'P12'],
 ['P00'],
 ['P8', 'P6'],
 ['P00'],
 ['P16', 'P13', 'P11'],
 ['P00', 'P12'],
 ['P16', 'P13', 'P11'],
 ['P12', 'P9', 'P1'],
 ['P12', 'P1', 'P15'],
 ['P12', 'P10', 'P5'],
 ['P8', 'P3', 'P12'],
 ['P8', 'P10', 'P3'],
 ['P12', 'P8', 'P15'],
 ['P16', 'P13', 'P11'],
 ['P00', 'P1'],
 ['P8'],
 ['P00', 'P1'],
 ['P13'],
 ['P12', 'P00', 'P15'],
 ['P16', 'P13', 'P

In [1186]:
def prob(list1):
    out=[]
    for i in list1:
        out.append(proba_test[i]-cut[i])
    return(out)

In [1187]:
cols = ID_test.iloc[:,1:].columns.values
mask = ID_test.iloc[:,1:].gt(0.0).values
out = [cols[x].tolist() for x in mask]

In [1188]:
# out

In [1189]:
# ID_test2=ID_test.copy()

In [1190]:
# ID_test2['Product_Holding_B2']=out

In [1191]:
# ID_test2['probablities']=ID_test2['Product_Holding_B2'].apply(prob)

In [1192]:
# ID_test2

In [1193]:
# S=[]
# for i in range(len(out)):
#     S.append(len(out[i]))

In [1194]:
# ID_test['length']=S

In [1195]:
# df4=ID_test[ID_test['length']==1]
# df5=ID_test[ID_test['length']!=1]

In [1196]:
# df5.shape

In [1197]:
# D

In [1198]:
# A=[]
# for i in range(ID_test.shape[0]):
#     B=[]
#     for j in range(ID_test.shape[1]):
#         if ID_test.iloc[i,j]==1:
#             B.append(ID_test.iloc[:,j:j+1].columns.to_list()[0][3:])
#     A.append(B)

In [1199]:
# for i in range(len(A)):
#     if len(A[i])==0:
#         A[i]=['P8']

In [1200]:
# for i in range(len(A)):
#     A[i]="A[i]"

In [1201]:
Submission=pd.DataFrame(columns=['Product_Holding_B2'])

In [1202]:
Submission['Product_Holding_B2']=S

In [1203]:
Submission

,Product_Holding_B2
0,"[P10, P4, P5]"
1,"[P8, P6, P1]"
2,"[P16, P13, P11]"
3,"[P12, P10, P4]"
4,"[P00, P1, P12]"
...,...
20322,"[P8, P15, P6]"
20323,"[P8, P6, P15]"
20324,"[P00, P9, P12]"
20325,"[P1, P6, P7]"


In [1204]:
Submission["Product_Holding_B2"]=Submission["Product_Holding_B2"].astype(str)

In [1205]:
# ID_train=pd.DataFrame(cust_ID.iloc[train.shape[0]:],columns=['Customer_ID'])

In [1206]:
Submission=pd.concat([test_ID,Submission],axis='columns')

In [1207]:
sub=pd.read_csv('sample_submission_kF044ur.csv')

In [1208]:
sub

,Customer_ID,Product_Holding_B2
0,CC372708,['P00']
1,CC216072,['P00']
2,CC387629,['P00']
3,CC389228,['P00']
4,CC394445,['P00']
...,...,...
20322,CC303542,['P00']
20323,CC266713,['P00']
20324,CC393639,['P00']
20325,CC285013,['P00']


In [1209]:
# sub.info()

In [1210]:
Submission

,Customer_ID,Product_Holding_B2
0,CC372708,"['P10', 'P4', 'P5']"
1,CC216072,"['P8', 'P6', 'P1']"
2,CC387629,"['P16', 'P13', 'P11']"
3,CC389228,"['P12', 'P10', 'P4']"
4,CC394445,"['P00', 'P1', 'P12']"
...,...,...
20322,CC303542,"['P8', 'P15', 'P6']"
20323,CC266713,"['P8', 'P6', 'P15']"
20324,CC393639,"['P00', 'P9', 'P12']"
20325,CC285013,"['P1', 'P6', 'P7']"


In [1211]:
Submission.to_csv('Submission.csv',index=False)

In [1212]:
Submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20327 entries, 0 to 20326
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer_ID         20327 non-null  object
 1   Product_Holding_B2  20327 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [1213]:
# Submission['Product_Holding_B2'].to_list()

In [971]:
# Submission['Product_Holding_B2']="['P6']"